<a href="https://colab.research.google.com/github/mohammadAliMkh/Pytorch/blob/main/food_vision_modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

os.mkdir("/content/food_vision_project")

In [2]:
%%writefile food_vision_project/fetch_data.py

import os
from pathlib import Path
import requests
from zipfile import ZipFile

def get_data(zip_file_name , raw_path_url):
  """
  Get zip file from url link

    args:
      zip_file_name: name of the zip file (include .zip)
      raw_path_url: url link that raw zip file (include .zip)
    
    return:
      train_dir: train direcotry path
      test_dir: test directory path
  """

  file_name = zip_file_name
  folder_name = file_name.split(".zip")[0]

  parent_dir = Path.cwd()
  data_dir = parent_dir/folder_name

  if Path.exists(data_dir):
    print(f"{folder_name} Folder Already Exists...")
  else:
    os.makedirs(data_dir)
    print(f"{folder_name} Folder Created...")



  # get pizza_steak_sushi.zip file from mrdbourrke github repository
  r = requests.get(raw_path_url)

  with open(file_name , "wb") as f:
    f.write(r.content)

  data_path = parent_dir/file_name
  print(f"{file_name} downloaded...")



  # loading the temp.zip and creating a zip object
  with ZipFile(data_path , 'r') as zObject:
    
      # Extracting all the members of the zip 
      # into a specific location.
      zObject.extractall(
          path=data_dir)
  print(f"{file_name} extracted at {data_dir}")
  # removing the downloaded.zip file from parent directory
  os.remove(data_path)

  train_dir = data_dir/"train"
  test_dir = data_dir/"test"

  return train_dir , test_dir

Writing food_vision_project/fetch_data.py


In [3]:
from food_vision_project import fetch_data
train_dir , test_dir = fetch_data.get_data("pizza_steak_sushi.zip" , "https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")

pizza_steak_sushi Folder Created...
pizza_steak_sushi.zip downloaded...
pizza_steak_sushi.zip extracted at /content/pizza_steak_sushi


In [10]:
%%writefile food_vision_project/data_setup.py
import torch
from torch.utils.data import DataLoader

from torchvision.datasets import ImageFolder
from torchvision import transforms

import os
from pathlib import Path


def create_dataLoader(train_dir:Path,
                      test_dir:Path,
                      batch_size:int,
                      transformer:transforms.Compose):
  '''
    Make test/train data Loder object

    inputs:
      train_dir: train data directory
      test_dir: test data directory
      batch_size: batch size of the dataLoader
      transformer: transforms.Compose object 
    
    return:
      train_dataLoader: torch.utils.data.DataLoader object
      test_dataLoader: torch.utils.data.DataLoader object
      class_names: list of the all class names
  '''

  BATCH_SIZE = batch_size
  TRAIN_DIR = train_dir
  TEST_DIR = test_dir
  NUM_WORKERS = os.cpu_count()

  TRANSFORMER = transformer



  train_dataset = ImageFolder(TRAIN_DIR , transform = TRANSFORMER)
  test_dataset = ImageFolder(TEST_DIR , transform = TRANSFORMER)


  train_dataLoader = DataLoader(train_dataset ,
                                batch_size = BATCH_SIZE,
                                shuffle = True,
                                num_workers = NUM_WORKERS,
                                pin_memory = True)

  test_dataLoader = DataLoader(test_dataset,
                              batch_size = BATCH_SIZE,
                              shuffle = False,
                              num_workers = NUM_WORKERS,
                              pin_memory=True)
  
  class_names = train_dataset.classes
  

  return train_dataLoader , test_dataLoader , class_names


Overwriting food_vision_project/data_setup.py


In [7]:
from torchvision import transforms

transfomer = transforms.Compose([
    transforms.Resize(size = (64 , 64)),
    transforms.ToTensor()
])

In [8]:
from food_vision_project import data_setup
x , y , z = data_setup.create_dataLoader(train_dir , test_dir , batch_size = 32 , transformer = transfomer)